#TF-IDF

---

##Importing necessary Libraries

In [23]:
# requirements
!pip install num2words

In [24]:
import re
import nltk
import math
import num2words
import pandas as pd;
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Tokenizer

In [25]:
porter_stemmer = nltk.stem.porter.PorterStemmer()


def tokenize_Words(sentences):
    punctuations=['+',',','?','!','/','@','#','$']
    tokens=[]
    for sentence in sentences:
        words = nltk.word_tokenize(sentence)	
        words = map(lambda x:x.lower(),words)
        filtered_words = [word for word in words if word not in nltk.corpus.stopwords.words('english')+punctuations]
        stemmed_word=[porter_stemmer.stem(word) for word in filtered_words]
        add_numbers=[num2words.num2words(int(words)) if words.isnumeric() else words for words in stemmed_word]
        tokens.append(add_numbers)
    return tokens



## Import Dataset
> Source: [Cricket Commentary - Kaggle](https://www.kaggle.com/shineucc/cricket-commentary)

In [26]:
Dataframe = pd.read_csv('dataset.csv')

Dataframe.head()

,Sentence
0,"Jarvis to Jahurul Islam,FOUR,Off the mark in s..."
1,"Jarvis to Jahurul Islam,1 run,some away swing ..."
2,"Jarvis to Shahriar Nafees,no run,short of a go..."
3,"Jarvis to Shahriar Nafees,no run,bowls it full..."
4,"Jarvis to Shahriar Nafees,no run,width from Ja..."


In [29]:
sentences = Dataframe.iloc[:, 0].values
sentences = [str(i) for i in sentences]
sentences = [i.replace(',', ' ') for i in sentences]

tokens = tokenize_Words(sentences)

## TF-IDF functions


In [30]:
def tf(t, d):
    return float(d.count(t)) / len(d)

def idf(t):
    c_doc = 0
    for d in D:
        if t in d:
            c_doc += 1			
    return 0 if c_doc == 0 else math.log(len(D)/float(c_doc))

def tf_idf(q, d, op):
    if op=='and':	
        if len(set(q).intersection(d))==len(set(q)):
            ans = 0
            for t in q:
                ans += (tf(t, d) * idf(t))			
            return  ans
        else:
            return 0
    else:
        ans = 0
        for t in q:
            ans += (tf(t, d) * idf(t))
        return  ans

##Driver Code

In [33]:
while True:
    query = input('\nenter query string or \'q\' to quit:\n')

    if query=='q':
        break

    if query.startswith('"'):
        op='and'	
    else:
        op='or'	

    query=re.sub(r'[#+?!"/@$,]',' ',query)
    query = tokenize_Words([query])

    scores = dict()
    i = 0

    for d in tokens:
        sc=tf_idf(query[0],d,op)	
        if sc!=0:
            scores[i] = sc
        i += 1

    if len(scores)==0:
        print ('Matches not Found')
    else:
        top_docs=sorted(scores, key=scores.get, reverse=True)[:10]
        rank = 1
        for doc in top_docs:
            print (f'\nrank: {rank} score: {scores[doc]}')
            print( sentences[doc].strip())
            rank+=1



enter query string or 'q' to quit:
driven through midwicket

rank: 1 score: 0.6440826208374175
Southee to Cook no run driven to cover

rank: 2 score: 0.6440826208374175
Southee to Trott no run driven into the covers

rank: 3 score: 0.5367355173645145
Broad to Rogers no run driven back down the pitch

rank: 4 score: 0.4600590148838696
Broad to Williamson no run full just outside off  driven to mid off

rank: 5 score: 0.4600590148838696
Bracewell to Cook no run full outside off and driven to mid-off

rank: 6 score: 0.4600590148838696
Wagner to Trott no run full on off stump  driven to cover

rank: 7 score: 0.4025516380233859
Swann to Brownlie no run tossed up and full  driven back to the bowler

rank: 8 score: 0.4025516380233859
Finn to McCullum no run full and just outside off  driven solidly to mid off

rank: 9 score: 0.4025516380233859
Williamson to Cook 1 run tossed up outside off  driven to cover

rank: 10 score: 0.4025516380233859
Wagner to Trott no run pushed across on off stump 